# Preparing 8p trans effects data for rSEA

## Setup

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
ttest_results_file = "8p_transeffects.tsv"

ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
ttest_results

,lscc_Database_ID,luad_Database_ID,ovarian_Database_ID,colon_pvalue,hnscc_pvalue,lscc_pvalue,luad_pvalue,ovarian_pvalue,colon_diff,hnscc_diff,lscc_diff,luad_diff,ovarian_diff
protein,,,,,,,,,,,,,
A1BG,NP_570602.2,NP_570602.2,NP_570602,0.827767,0.884785,0.768015,0.248965,0.829152,-0.119042,-0.037599,0.312395,-0.442692,-0.294055
A2M,NP_000005.2|NP_001334353.1|NP_001334354.1,NP_000005.2|NP_001334353.1|NP_001334354.1|K4JD...,NP_000005,0.865747,0.794917,0.849859,0.385338,0.942809,-0.081407,-0.094712,0.301145,-0.461877,-0.244195
AAAS,NP_056480.1|NP_001166937.1,NP_056480.1|NP_001166937.1,NP_056480,0.664097,0.902739,0.949722,0.271582,0.995675,-0.100606,0.020226,-0.037585,0.219718,0.030730
AACS,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417,0.693473,0.915573,0.729842,0.907597,0.968689,0.089959,0.039000,-0.527445,-0.103594,0.132034
AAGAB,NP_078942.3|NP_001258814.1,NP_078942.3|NP_001258814.1,NP_078942,0.751486,0.924353,0.834289,0.734112,0.995675,-0.078345,-0.026190,-0.174586,-0.119763,0.049414
AAK1,NP_055726.3,NaN,NP_055726,0.903792,0.955279,0.908026,NaN,0.944506,0.025289,-0.011070,0.081181,NaN,0.104969
AAMDC,NP_001303889.1|NP_001350493.1,NP_001303889.1|NP_001350493.1|NP_001303886.1|N...,NP_078960,0.994133,0.880055,0.967353,0.952731,0.983804,-0.003264,0.052767,0.070351,-0.050954,0.126284
AAMP,NP_001289474.1|NP_001078.2,NP_001289474.1|NP_001078.2,NP_001078,0.662087,0.844905,0.730621,0.406354,0.968689,-0.085926,0.048081,0.328295,-0.276061,0.091424
AAR2,NP_001258803.1,NP_001258803.1,NP_001258803,0.456258,0.955532,0.826538,0.845902,0.995675,0.119473,-0.014591,-0.148427,0.062387,0.039881


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p,change
0,colon,A1BG,NaN,0.827767,-0.119042
1,colon,A2M,NaN,0.865747,-0.081407
2,colon,AAAS,NaN,0.664097,-0.100606
3,colon,AACS,NaN,0.693473,0.089959
4,colon,AAGAB,NaN,0.751486,-0.078345
5,colon,AAK1,NaN,0.903792,0.025289
6,colon,AAMDC,NaN,0.994133,-0.003264
7,colon,AAMP,NaN,0.662087,-0.085926
8,colon,AAR2,NaN,0.456258,0.119473
9,colon,AARS2,NaN,0.322802,0.138317


## Select the proteins with a significant change

In [6]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)

## Find how many cancers each protein was different in

In [7]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist())
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,num_cancers
protein,,
PCM1,"[colon, hnscc, lscc, luad]",4
XPO7,"[colon, hnscc, lscc, luad]",4
PPP2CB,"[colon, lscc, luad]",3
PPP2R2A,"[colon, lscc, luad]",3
AGPAT5,"[hnscc, lscc, luad]",3
CNOT7,"[hnscc, lscc, luad]",3
ERI1,"[hnscc, lscc, luad]",3
VPS37A,"[hnscc, lscc, luad]",3
FDFT1,"[colon, lscc]",2


## Info for most common proteins

- [PCM1](https://www.uniprot.org/uniprot/Q15154)    (colon, hnscc, lscc, luad)  4
- [XPO7](https://www.uniprot.org/uniprot/Q9UIA9)    (colon, hnscc, lscc, luad)  4
- [PPP2CB](https://www.uniprot.org/uniprot/P62714)  (colon, lscc, luad)     3
- [PPP2R2A](https://www.uniprot.org/uniprot/P63151)     (colon, lscc, luad)     3
- [AGPAT5](https://www.uniprot.org/uniprot/Q9NUQ2)  (hnscc, lscc, luad)     3
- [CNOT7](https://www.uniprot.org/uniprot/Q9UIV1)   (hnscc, lscc, luad)     3
- [ERI1](https://www.uniprot.org/uniprot/Q8IV48)    (hnscc, lscc, luad)     3
- [VPS37A](https://www.uniprot.org/uniprot/Q8NEZ2)  (hnscc, lscc, luad)     3
